#### Precipitation and Air Temperature data were downloaded from
- NASA's [Daymet](https://daymet.ornl.gov/) using coordinates from the Clemson University Pee Dee Research Center
- latitude: 34.289	
- longitude: -79.737

#### Wind and humidity data were downloaded from
- Climate [Engine](https://clim-engine.appspot.com/climateEngine) using coordinates from the Clemson Pee Dee Research and Education Center
- latitude: 34.289	
- longitude: -79.737

#### These are more precise environmental data which will be used in combination with the trait [data](https://github.com/MagicMilly/for-data-publication/tree/main/data/processed) from Brenton et al. 2016 at the Clemson University Pee Dee Research Center

In [5]:
import datetime
import numpy as np
import pandas as pd

### A. Daymet Download

In [7]:
daymet_0 = pd.read_csv('data/raw/clemson_raw_daily_2014_updated.csv')
print(daymet_0.shape)
# daymet_0.head()

(365, 9)


### Columns from Daymet Download
- year
- day of year
- dayl - daylength, seconds per day
- precipitation in mm
- srad - shortwave radiation (can be used with dayl to calculate total daily radiation)
- swe - snow water equivalent
- air temperature max in C
- air temperature min in C
- vp - daily average partial pressure of water vapor, in Pascals

#### Add Dates

In [8]:
date_range_2014 = pd.date_range(start='2014-01-01', end='2014-12-31')

In [9]:
daymet_1 = daymet_0.copy()
daymet_1['date'] = date_range_2014
print(daymet_1.shape)
# daymet_1.tail()

(365, 10)


#### Drop Columns
- year
- dayl
- solar radiation
- snow water equivalent

In [10]:
cols_to_drop = ['year', 'dayl (s)', 'srad (W/m^2)', 'swe (kg/m^2)']

daymet_2 = daymet_1.drop(labels=cols_to_drop, axis=1)
print(daymet_2.shape)
# daymet_2.head()

(365, 6)


#### Rename Columns

In [11]:
daymet_3 = daymet_2.rename({'yday': 'day_of_year', 'prcp (mm/day)': 'precip_mm', 'tmax (deg c)': 'temp_max_c',
                           'tmin (deg c)': 'temp_min_c', 'vp (Pa)': 'vapor_pressure'}, axis=1)

print(daymet_3.shape)
# daymet_3.tail()

(365, 6)


In [12]:
# daymet_3.dtypes

### B. Climate Engine Download
- rh_n - dataset with relative humidity values
- wind_n - dataset with wind values
    - Assumption that these are average wind speed values per day, but have not found the official documentation yet

In [13]:
rh_0 = pd.read_csv('data/raw/clemson_rh_2014.csv')
print(rh_0.shape)
# rh_0.head()

(365, 3)


#### Rename Columns

In [14]:
# rh_0.columns

In [15]:
rh_1 = rh_0.rename({'(%) Min Rel. Humidity (gridMET), -79.7370E,34.2890N ,2014-01-01 to 2014-12-31': 'min_rh',
                   '(%) Max Rel. Humidity (gridMET), -79.7370E,34.2890N ,2014-01-01 to 2014-12-31': 'max_rh'}, axis=1)

print(rh_1.shape)
# rh_1.head()

(365, 3)


#### Change `DateTime` values to datetime objects

In [16]:
date_range = pd.to_datetime(rh_1.DateTime)
# date_range

In [17]:
rh_2 = rh_1.copy()
rh_2['date'] = date_range
print(rh_2.dtypes)
# rh_2.head()

DateTime            object
min_rh             float64
max_rh             float64
date        datetime64[ns]
dtype: object


#### Drop DateTime Column

In [18]:
rh_3 = rh_2.drop('DateTime', axis=1)
print(rh_3.shape)
# rh_3.tail()

(365, 3)


In [19]:
wind_0 = pd.read_csv('data/raw/climate_engine_wind_speed_2014.csv')
print(wind_0.shape)
# wind_0.head()

(365, 2)


#### Rename Column

In [20]:
wind_1 = wind_0.rename({'(m/s) Wind Speed (gridMET) at -79.7370E,34.2890N, 2014-01-01 to 2014-12-31': 'wind_speed'},
                      axis=1)

print(wind_1.shape)
# wind_1.head()

(365, 2)


#### New Column with datetime object

In [21]:
date_range = pd.to_datetime(wind_1.DateTime)

wind_2 = wind_1.copy()
wind_2['date'] = date_range

print(wind_2.shape)
# wind_2.tail()

(365, 3)


#### Drop DateTime Column

In [22]:
wind_3 = wind_2.drop('DateTime', axis=1)
# wind_3.head()

#### Merge DataFrames

In [23]:
temp_rh_wind_0 = daymet_3.merge(rh_3, how='outer', right_on='date', left_on='date').merge(wind_3, how='outer',
                                                                                    left_on='date', right_on='date')

print(temp_rh_wind_0.shape)
# temp_rh_wind_0.head()

(365, 9)


#### Add cumulative precipitation column

In [24]:
temp_rh_wind_1 = temp_rh_wind_0.copy()

temp_rh_wind_1['cum_precip_mm'] = np.cumsum(temp_rh_wind_1.precip_mm)
print(temp_rh_wind_1.shape)
# temp_rh_wind_1.head(10)

(365, 10)


#### Round values to 2 decimal points

In [25]:
temp_rh_wind_2 = temp_rh_wind_1.copy()

temp_rh_wind_2[['temp_max_c', 'temp_min_c', 'min_rh', 'max_rh', 'wind_speed', 
               'cum_precip_mm']] = temp_rh_wind_2[['temp_max_c', 'temp_min_c', 'min_rh', 'max_rh', 
                                                 'wind_speed', 'cum_precip_mm']].round(2)

print(temp_rh_wind_2.shape)
# temp_rh_wind_2.tail()

(365, 10)


#### Reorder Columns

In [26]:
# temp_rh_wind_2.columns

In [29]:
new_col_order = ['date', 'day_of_year', 'temp_min_c', 'temp_max_c', 'min_rh', 'max_rh', 'vapor_pressure', 
                 'wind_speed', 'precip_mm', 'cum_precip_mm']

temp_rh_wind_3 = pd.DataFrame(data=temp_rh_wind_2, columns=new_col_order)
print(temp_rh_wind_3.shape)
temp_rh_wind_3.head(3)

(365, 10)


,date,day_of_year,temp_min_c,temp_max_c,min_rh,max_rh,vapor_pressure,wind_speed,precip_mm,cum_precip_mm
0,2014-01-01,1,0.5,12.0,39.86,64.12,640,2.14,0,0
1,2014-01-02,2,3.0,10.0,64.53,100.00,760,1.52,5,5
2,2014-01-03,3,-3.5,10.0,56.70,100.00,480,7.26,11,16


#### Write to CSV

In [30]:
temp_rh_wind_3.to_csv('data/interim/updated_clemson_weather_2014_2020-06-24.csv', index=False)